In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas_datareader import data as web
import numpy as np

In [1]:
# Make function for calls to Yahoo Finance
def get_adj_close(ticker, start, end):
'''
A function that takes ticker symbols, starting period, ending period
as arguments and returns with a Pandas DataFrame of the Adjusted Close Prices
for the tickers from Yahoo Finance
'''
start = start
end = end
info = web.DataReader(ticker, data_source='yahoo', start=start, end=end)['Adj Close']
return pd.DataFrame(info)

def Bollinger_Bands(item):
    # Calculate 30 Day Moving Average, Std Deviation, Upper Band and Lower Band
    item['30 Day MA'] = item['Adj Close'].rolling(window=20).mean()
    # set .std(ddof=0) for population std instead of sample
    item['30 Day STD'] = item['Adj Close'].rolling(window=20).std()
    item['Upper Band'] = item['30 Day MA'] + (item['30 Day STD'] * 2)
    item['Lower Band'] = item['30 Day MA'] - (item['30 Day STD'] * 2)
    # Simple 30 Day Bollinger Band for Facebook (2016-2017)
    item[['Adj Close', '30 Day MA', 'Upper Band', 'Lower Band']].plot(figsize=(12,6))
    plt.title('30 Day Bollinger Band for Facebook')
    plt.ylabel('Price (USD)')
    plt.show();

In [ ]:
import talib
from logic import MarketTrend
from logic import Indicator
from logic.candle import Candle
class ChandelierExit(Indicator):
def __init__(self):
    super(ChandelierExit,self).__init__()
    self.enter_period = 22
    self.C1 = 0.0
    self.C2 = 0.0
    self._high = []
    self._low = []
    self._close = []
    self.trend = MarketTrend.NONE
def Update(self, datapoint):
    if not isinstance(datapoint, Candle):
    return
# append data to storage
    self._high.append(datapoint.High)
    self._low.append(datapoint.Low)
    self._close.append(datapoint.Close)
    if (len(self._high) > self.enter_period):
        self._high.pop(0)
        self._low.pop(0)
        self._close.pop(0)
# find H and L enter_period max and min
    Pmax = max(self._high)
    Pmin = min(self._low)
    7
# calculate enter_period Average True Range
    ATR = talib.ATR(numpy.array(self._high), numpy.array(self._low), numpy.array(self._close),
    timeperiod=self.enter_period)[-1]
# Chandelier's Exit
    self.C1 = Pmax - 3.0 * ATR
    self.C2 = Pmin + 3.0 * ATR
    self.trend = MarketTrend.NONE
    if (self.C1 > datapoint.Close):
        self.trend = MarketTrend.EXIT_LONG
    if (self.C2 < datapoint.Close):
        self.trend = MarketTrend.EXIT_SHORT
def AmountOfDataStillMissing(self):
    return max(0, self.enter_period - len(self._high))
def SeenEnoughData(self):
    return (self.AmountOfDataStillMissing == 0)

In [ ]:
import pandas as pd
from pandas_datareader import data, wb
import matplotlib as mpl
from mpl_finance import candlestick_ohlc
import matplotlib.dates as dates
import datetime
import matplotlib.pyplot as plt
def Ichimoku_Cloud(ticker, start, end):
    d=data.DataReader(ticker, 'yahoo', start, end)
    # convert dates to integer values for later use with the
    # matplotlib candlestick charting function (it doesn't accept datetimes)
    d['Dates'] = dates.date2num(d.index)
    # Tenkan-sen (Conversion Line): (9-period high + 9-period low)/2))
    nine_period_high = d['High'].rolling(window= 9).max()
    nine_period_low = d['Low'].rolling(window= 9).min()
    d['tenkan_sen'] = (nine_period_high + nine_period_low) /2
    # Kijun-sen (Base Line): (26-period high + 26-period low)/2))
    period26_high = d['High'].rolling(window=26).max()
    period26_low = d['Low'].rolling(window=26).min()
    d['kijun_sen'] = (period26_high + period26_low) / 2
    # Senkou Span A (Leading Span A): (Conversion Line + Base Line)/2))
    d['senkou_span_a'] = ((d['tenkan_sen'] + d['kijun_sen']) / 2).shift(26)
    # Senkou Span B (Leading Span B): (52-period high + 52-period low)/2))
    period52_high = d['High'].rolling(window=52).max()
    period52_low = d['Low'].rolling(window=52).min()
    d['senkou_span_b'] = ((period52_high + period52_low) / 2).shift(26)
    # The most current closing price plotted 26 time periods behind (optional)
    d['chikou_span'] = d['Close'].shift(-26)
    # create a quick plot of the results to see what we have created
    d.drop(['Dates', 'Volume'], axis=1).plot(figsize=(15,8))
    plt.show()
    # Re-arrange data so that each row contains values of a day: 'Date','Open','High','Low','Close'.
    # The 'Date' can't be a "datetime" object as the function wont accept them. That's why we converted
    # our 'Date' column into integer values using the "date2num" function earlier.
    quotes = [tuple(x) for x in d[['Dates','Open','High','Low','Close']].values]
    # Plot candlestick, along with Ichimoku elements
    fig, ax = plt.subplots(figsize=(15,8))
    d[['tenkan_sen','kijun_sen','senkou_span_a','senkou_span_b','chikou_span']].plot(ax=ax, linewidth=0.5)
    candlestick_ohlc(ax, quotes, width=1.0, colorup='g', colordown='r');

In [ ]:
d = get_adj_close(ticker, start, end)
def KAMA(x, n=10, pow1=2, pow2=30):
    ''' kama indicator '''
    ''' accepts pandas dataframe of prices '''
    d['absDiffx'] = abs(x - x.shift(1) )
    d['ER.num'] = ( x - x.shift(n) )
    d['ER.den'] = pandas.stats.moments.rolling_sum(d['absDiffx'],n)
    d['ER'] = d['ER.num'] / d['ER.den']
    d['SC'] = ( d['ER']*(2.0/(pow1+1)-2.0/(pow2+1.0))+2/(pow2+1.0) ) ** 2.0
    return d['SC']
kama = KAMA(d.Close, n=10, pow1=2, pow2=30)
df = pd.DataFrame({'price':d.Close, 'KAMA':KAMA(d.Close, n=10, pow1=2, pow2=30) })
df.plot(subplots=True)

In [ ]:
def KELCH(df, n):
    KelChM = pd.Series(pd.rolling_mean((df['High'] + df['Low'] + df['Close']) / 3, n), name = 'KelChM_' +
    str(n))
    KelChU = pd.Series(pd.rolling_mean((4 * df['High'] - 2 * df['Low'] + df['Close']) / 3, n), name =
    'KelChU_' + str(n))
    KelChD = pd.Series(pd.rolling_mean((-2 * df['High'] + 4 * df['Low'] + df['Close']) / 3, n), name =
    'KelChD_' + str(n))
    df = df.join(KelChM)
    df = df.join(KelChU)
    df = df.join(KelChD)
    return df
data1 = pd.read_csv('E:/MarketData/EURUSD60.csv', header=None)
data1.columns=['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume']
#data1=ATR(data1,14)
data1=KELCH(data1,14)
data1['KelChM_14'].plot(figsize=(12,8));
data1['KelChU_14'].plot(figsize=(12,8));
data1['KelChD_14'].plot(figsize=(12,8));
plt.show()

In [ ]:
import pyEX as p
ticker = 'AMD'
timeframe = '1y'
df = p.chartDF(ticker, timeframe)
df = df[['close']]
df.reset_index(level=0, inplace=True)
df.columns=['ds','y']
plt.plot(df.ds, df.y)
plt.show()

# simple moving average
rolling_mean = df.y.rolling(window=20).mean()
rolling_mean2 = df.y.rolling(window=50).mean()plt.plot(df.ds, df.y, label='AMD')
plt.plot(df.ds, rolling_mean, label='AMD 20 Day SMA', color='orange')
plt.plot(df.ds, rolling_mean2, label='AMD 50 Day SMA', color='magenta')
plt.legend(loc='upper left')
plt.show()
# exponential moving average (EMA)
exp1 = df.y.ewm(span=20, adjust=False).mean()
exp2 = df.y.ewm(span=50, adjust=False).mean()plt.plot(df.ds, df.y, label='AMD')
plt.plot(df.ds, exp1, label='AMD 20 Day EMA')
plt.plot(df.ds, exp2, label='AMD 50 Day EMA')

plt.legend(loc='upper left')
plt.show()

In [ ]:
from __future__ import (absolute_import, division, print_function, unicode_literals)
import sys
from . import Indicator, MovingAverage

class EnvelopeMixIn(object):
'''
MixIn class to create a subclass with another indicator. The main line of
that indicator will be surrounded by an upper and lower band separated a
given "perc"entage from the input main line
The usage is:
- Class XXXEnvelope(XXX, EnvelopeMixIn)
Formula:
- 'line' (inherited from XXX))
- top = 'line' * (1 + perc)
- bot = 'line' * (1 - perc)
See also:
-
http://stockcharts.com/school/doku.php?id=chart_school:technical_indicators:moving_average_envelopes
'''
    lines = ('top', 'bot',)
    params = (('perc', 2.5),)
    plotlines = dict(top=dict(_samecolor=True), bot=dict(_samecolor=True),)
    def __init__(self):
        # Mix-in & directly from object -> does not necessarily need super
        # super(EnvelopeMixIn, self).__init__()
        perc = self.p.perc / 100.0
        self.lines.top = self.lines[0] * (1.0 + perc)
        self.lines.bot = self.lines[0] * (1.0 - perc)
        super(EnvelopeMixIn, self).__init__()
class _EnvelopeBase(Indicator):
    lines = ('src',)
    # plot the envelope lines along the passed source
    plotinfo = dict(subplot=False)
    # Do not replot the data line
    plotlines = dict(src=dict(_plotskip=True))
    def __init__(self):
        self.lines.src = self.data
        super(_EnvelopeBase, self).__init__()
class Envelope(_EnvelopeBase, EnvelopeMixIn):
    for movav in MovingAverage._movavs[1:]:
    _newclsdoc = '''
    %s and envelope bands separated "perc" from it
    Formula:
    - %s (from %s)
    - top = %s * (1 + perc)
    - bot = %s * (1 - perc)
    See also:
    -
    http://stockcharts.com/school/doku.php?id=chart_school:technical_indicators:moving_average_envelopes
    '''
    # Skip aliases - they will be created automatically
    if getattr(movav, 'aliased', ''):
        continue
    movname = movav.__name__
    linename = movav.lines._getlinealias(0)
    newclsname = movname + 'Envelope'
    newaliases = []
    for alias in getattr(movav, 'alias', []):
        for suffix in ['Envelope']:
            newaliases.append(alias + suffix)
    newclsdoc = _newclsdoc % (movname, linename, movname, linename, linename)
    newclsdct = {'__doc__': newclsdoc,
    '__module__': EnvelopeMixIn.__module__,
    '_notregister': True,
    'alias': newaliases}
    newcls = type(str(newclsname), (movav, EnvelopeMixIn), newclsdct)
    module = sys.modules[EnvelopeMixIn.__module__]
    setattr(module, newclsname, newcls)

In [ ]:
def psar(barsdata, iaf = 0.02, maxaf = 0.2):
    length = len(barsdata)
    dates = list(barsdata['Date'])
    high = list(barsdata['High'])
    low = list(barsdata['Low'])
    close = list(barsdata['Close'])
    psar = close[0:len(close)]
    psarbull = [None] * length
    psarbear = [None] * length
    bull = True
    af = iaf
    ep = low[0]
    hp = high[0]
    lp = low[0]
    for i in range(2,length):
        if bull:
            psar[i] = psar[i - 1] + af * (hp - psar[i - 1])
        else:
            psar[i] = psar[i - 1] + af * (lp - psar[i - 1])
    reverse = False
    if bull:
        if low[i] < psar[i]:
            bull = False
            reverse = True
            psar[i] = hp
            lp = low[i]
            af = iaf
    else:
        if high[i] > psar[i]:
            bull = True
            reverse = True
            psar[i] = lp
            hp = high[i]
            af = iaf
            
    if not reverse:
        if bull:
            if high[i] > hp:
                hp = high[i]
                af = min(af + iaf, maxaf)
            if low[i - 1] < psar[i]:
                psar[i] = low[i - 1]
            if low[i - 2] < psar[i]:
                psar[i] = low[i - 2]
        else:
            if low[i] < lp:
                lp = low[i]
                af = min(af + iaf, maxaf)
            if high[i - 1] > psar[i]:
                psar[i] = high[i - 1]
            if high[i - 2] > psar[i]:
                psar[i] = high[i - 2]
    if bull:
        psarbull[i] = psar[i]
    else:
        psarbear[i] = psar[i]
    
    return {"dates":dates, "high":high, "low":low, "close":close, "psar":psar,
"p sarbear":psarbear, "psarbull":psarbull}
import sys
import os
if len(sys.argv) < 2:
    sys.exit("Usage: %s datafile.csv" % sys.argv[0])
if not os.path.exists(sys.argv[1]):
    sys.exit("Error: can't open file '%s': No such file" % sys.argv[1])
barsdata = pd.read_csv(sys.argv[1])
#Reindex the data: ascending dates are expected in the function
barsdata = barsdata.reindex(index=barsdata.index[::-1])
#Convert strings to actual timestamps
barsdata['Date'] = [datetime.strptime(x, '%Y-%m-%d') for x in barsdata['Date']]
startidx = 0
endidx = len(barsdata)
result = psar(barsdata)
dates = result['dates'][startidx:endidx]
close = result['close'][startidx:endidx]
psarbear = result['psarbear'][startidx:endidx]
psarbull = result['psarbull'][startidx:endidx]
plt.plot(dates, close)
plt.plot(dates, psarbull)
plt.plot(dates, psarbear)
plt.grid()
plt.show()

In [ ]:
from __future__ import (absolute_import, division, print_function,
unicode_literals)
from . import Indicator, CmpEx
class PivotPoint(Indicator):
    '''
    Defines a level of significance by taking into account the average of price
    bar components of the past period of a larger timeframe. For example when
    operating with days, the values are taking from the already "past" month
    fixed prices.
    Example of using this indicator:
    data = btfeeds.ADataFeed(dataname=x, timeframe=bt.TimeFrame.Days)
    cerebro.adddata(data)
    cerebro.resampledata(data, timeframe=bt.TimeFrame.Months)
    In the ``__init__`` method of the strategy:
    pivotindicator = btind.PivotPoiont(self.data1) # the resampled data
    The indicator will try to automatically plo to the non-resampled data. To
    disable this behavior use the following during construction:
    - _autoplot=False
    Note:
    The example shows *days* and *months*, but any combination of timeframes
    can be used. See the literature for recommended combinations
    Formula:
    - pivot = (h + l + c) / 3 # variants duplicate close or add open
    - support1 = 2.0 * pivot - high
    - support2 = pivot - (high - low)
    - resistance1 = 2.0 * pivot - low
    - resistance2 = pivot + (high - low)
    See:
    - http://stockcharts.com/school/doku.php?id=chart_school:technical_indicators:pivot_points
    - https://en.wikipedia.org/wiki/Pivot_point_(technical_analysis)
    '''
    lines = ('p', 's1', 's2', 'r1', 'r2',)
    plotinfo = dict(subplot=False)
    params = (
    ('open', False), # add opening price to the pivot point
    ('close', False), # use close twice in the calcs
    ('_autoplot', True), # attempt to plot on real target data
    )
    def _plotinit(self):
        # Try to plot to the actual timeframe master
        if self.p._autoplot:
        if hasattr(self.data, 'data'):
        self.plotinfo.plotmaster = self.data.data
    def __init__(self):
        o = self.data.open
        h = self.data.high # current high
        l = self.data.low # current low
        c = self.data.close # current close
        if self.p.close:
            self.lines.p = p = (h + l + 2.0 * c) / 4.0
        elif self.p.open:
            self.lines.p = p = (h + l + c + o) / 4.0
        else:
            self.lines.p = p = (h + l + c) / 3.0
        self.lines.s1 = 2.0 * p - h
        self.lines.r1 = 2.0 * p - l
        self.lines.s2 = p - (h - l)
        self.lines.r2 = p + (h - l)
        super(PivotPoint, self).__init__() # enable coopertive inheritance
        if self.p._autoplot:
            self.plotinfo.plot = False # disable own plotting
            self() # Coupler to follow real object

In [ ]:
#Price Channels
def get_high(stock, start, end):
    return web.get_data_yahoo(stock, start, end)['High']
def get_low(stock, start, end):
    return web.get_data_yahoo(stock, start, end)['Low']
def get_close(stock, start, end):
    return web.get_data_yahoo(stock, start, end)['Adj Close']
x = pd.DataFrame(get_high('FB', '1/1/2016', '12/31/2016'))
x['Low'] = pd.DataFrame(get_low('FB', '1/1/2016', '12/31/2016'))
x['Close'] = pd.DataFrame(get_close('FB', '1/1/2016', '12/31/2016'))
x['4WH'] = pd.rolling_max(x['High'], 20)
x['4WL'] = pd.rolling_min(x['Low'], 20)
x['50 sma'] = pd.rolling_mean(x['Close'], 50)
x.plot(y=['4WH', '4WL', '50 sma', 'Close'])

In [ ]:
#Price by Volume Chart (PBV)
import quandl
from matplotlib.ticker import FuncFormatter
import argparse

def parse_args():
    parser = argparse.ArgumentParser(description='Volume at Price Chart')
    parser.add_argument('qcode', help='The Quandle code, including database \
    prefix e.g. WIKI/F of the instrument being tested')
    parser.add_argument('start', help='Starting date in YYYY-MM-DD format')
    parser.add_argument('end', help='Ending date in YYYY-MM-DD format')
    parser.add_argument('round', type=int, help='Round to the Nearest')
    return parser.parse_args()
def get_quandl_data(ticker, start, end):
    data = quandl.get(ticker, start_date=start, end_date=end)
    return data
def custom_round(x, base=5):
    return int(base * round(float(x)/base))
def round_and_group(df, base=5):
    # https://stackoverflow.com/questions/40372030/pandas-round-to-the-nearest-n
    # Extract the data we want
    df = data[['Close', 'Volume']].copy()
    # Round to nearest X
    df['Close'] = df['Close'].apply(lambda x: custom_round(x, base=base))
    # Remove the date index
    df = df.set_index('Close')
    df = df.groupby(['Close']).sum()
    return df
def thousands(x, pos):
    'The two args are the value and tick position'
    return '%1.0fK' % (x*1e-3)
def create_plot(x_series, x_label, y_pos, y_tick_labels, colour, title):
    # Excample horizontal bar chart code taken from:
    # https://matplotlib.org/gallery/lines_bars_and_markers/barh.html
    plt.rcdefaults()
    fig, ax = plt.subplots()
    ax.barh(y_pos, x_series, align='center',color=colour)
    ax.set_yticks(y_pos)
    ax.set_yticklabels(y_tick_labels)
    formatter = FuncFormatter(thousands)
    ax.xaxis.set_major_formatter(formatter)
    ax.set_xlabel(x_label)
    ax.set_title(title)
    plt.xticks(rotation=325)
    plt.show()

# Setup api
quandl_api = "INSERT YOUR API KEY"
quandl.ApiConfig.api_key = quandl_api
#Get Args
args = parse_args()
print("VOLUME AT PRICE")
print("QCODE: {}".format(args.qcode))
print("START: {}".format(args.start))
print("END: {}".format(args.end))
# Get data
22
data = get_quandl_data(args.qcode, args.start, args.end)
# Massage the data
data = round_and_group(data, base=args.round)
# Prepare data for the chart
y_pos = np.arange(len(data.index.values))
# Get the chart
plt = create_plot(
data['Volume'], # x_series
'Volume', # x_label
y_pos, # Y positioning
data.index.values, # y_tick_labels
'Green', # Bar color
'VOLUME AT PRICE: {}'.format(args.qcode) # Title
)

In [ ]:
from clr import AddReference
AddReference("System")
AddReference("QuantConnect.Common")
AddReference("QuantConnect.Indicators")
AddReference("QuantConnect.Algorithm")
AddReference("QuantConnect.Algorithm.Framework")
from System import *
from QuantConnect import *
from QuantConnect.Indicators import *
from QuantConnect.Data import *
from QuantConnect.Data.Market import *
from QuantConnect.Orders import *
from QuantConnect.Algorithm import *
from QuantConnect.A lgorithm.Framework import *
from QuantConnect.Algorithm.Framework.Execution import *
from QuantConnect.Algorithm.Framework.Portfolio import *

class VolumeWeightedAveragePriceExecutionModel(ExecutionModel):
'''Execution model that submits orders while the current market price is more
favorable that the current volume weighted average price.'''
    def __init__(self):
        '''Initializes a new instance of the
        VolumeWeightedAveragePriceExecutionModel class'''
        self.targetsCollection = PortfolioTargetCollection()
        self.symbolData = {}
        # Gets or sets the maximum order quantity as a percentage of the current bar's
        volume.
        # This defaults to 0.01m = 1%. For example, if the current bar's volume is 100,
        # then the maximum order size would equal 1 share.
        self.MaximumOrderQuantityPercentVolume = 0.01
    def Execute(self, algorithm, targets):
        '''Executes market orders if the standard deviation of price is more
        than the configured number of deviations in the favorable direction.
        Args:
        algorithm: The algorithm instance
        targets: The portfolio targets'''
        # update the complete set of portfolio targets with the new targets
        self.targetsCollection.AddRange(targets)
        # for performance we check count value, OrderByMarginImpact and
        ClearFulfilled are expensive to call
        if self.targetsCollection.Count > 0:
            for target in self.targetsCollection.OrderByMarginImpact(algorithm):
                symbol = target.Symbol
        # calculate remaining quantity to be ordered
                unorderedQuantity = OrderSizing.GetUnorderedQuantity(algorithm, target)
        # fetch our symbol data containing our VWAP indicator
                data = self.symbolData.get(symbol, None)
                if data is None: return
        # check order entry conditions
                if self.PriceIsFavorable(data, unorderedQuantity):
        # adjust order size to respect maximum order size based on a percentage of current volume
                    orderSize = OrderSizing.GetOrderSizeForPercentVolume(data.Security,self.MaximumOrderQuantityPercentVolume, unorderedQuantity)
                if orderSize != 0:
                    algorithm.MarketOrder(symbol, orderSize)
        self.targetsCollection.ClearFulfilled(algorithm)
    def OnSecuritiesChanged(self, algorithm, changes):
    '''Event fired each time the we add/remove securities from the data feed
    Args:
    algorithm: The algorithm instance that experienced the change in securities
    changes: The security additions and removals from the algorithm'''
        for removed in changes.RemovedSecurities:
        # clean up removed security data
            if removed.Symbol in self.symbolData:
                if self.IsSafeToRemove(algorithm, removed.Symbol):
                    data = self.symbolData.pop(removed.Symbol)
                        algorithm.SubscriptionManager.RemoveConsolidator(removed.Symbol,data.Consolidator)
        for added in changes.AddedSecurities:
            if added.Symbol not in self.symbolData:
                self.symbolData[added.Symbol] = SymbolData(algorithm, added)
    def PriceIsFavorable(self, data, unorderedQuantity):
        '''Determines if the current price is more than the configured
        number of standard deviations away from the mean in the favorable direction.'''
        if unorderedQuantity > 0:
            if data.Security.BidPrice < data.VWAP:
                return True
        else:
            if data.Security.AskPrice > data.VWAP:
                return True
        return False
    def IsSafeToRemove(self, algorithm, symbol):
    '''Determines if it's safe to remove the associated symbol data'''
    # confirm the security isn't currently a member of any universe
        return not any([kvp.Value.ContainsMember(symbol) for kvp in algorithm.UniverseManager])
class SymbolData:
    def __init__(self, algorithm, security):
        self.Security = security
        self.Consolidator = algorithm.ResolveConsolidator(security.Symbol, security.Resolution)
        name = algorithm.CreateIndicatorName(security.Symbol, "VWAP", security.Resolution)
        self.vwap = IntradayVwap(name)
        algorithm.RegisterIndicator(security.Symbol, self.vwap, self.Consolidator)
    @property
    def VWAP(self):
        return self.vwap.Value
class IntradayVwap:
    '''Defines the canonical intraday VWAP indicator'''
    def __init__(self, name):
        self.Name = name
        self.Value = 0.0
        self.lastDate = datetime.min
        self.sumOfVolume = 0.0
        self.sumOfPriceTimesVolume = 0.0
    @property
    def IsReady(self):
        return self.sumOfVolume > 0.0
    def Update(self, input):
        '''Computes the new VWAP'''
        success, volume, averagePrice = self.GetVolumeAndAveragePrice(input)
        if not success:
            return self.IsReady
        # reset vwap on daily boundaries
        if self.lastDate != input.EndTime.date():
            self.sumOfVolume = 0.0
            self.sumOfPriceTimesVolume = 0.0
            self.lastDate = input.EndTime.date()
        # running totals for Σ PiVi / Σ Vi
        self.sumOfVolume += volume
        self.sumOfPriceTimesVolume += averagePrice * volume
        if self.sumOfVolume == 0.0:
        # if we have no trade volume then use the current price as VWAP
            self.Value = input.Value
            return self.IsReady
        self.Value = self.sumOfPriceTimesVolume / self.sumOfVolume
        return self.IsReady
    def GetVolumeAndAveragePrice(self, input):
    '''Determines the volume and price to be used for the current input in the
    VWAP computation'''
    if type(input) is Tick:
        if input.TickType == TickType.Trade:
            return True, float(input.Quantity), float(input.LastPrice)
    if type(input) is TradeBar:
        if not input.IsFillForward:
            averagePrice = float(input.High + input.Low + input.Close) / 3
            return True, float(input.Volume), averagePrice
    return False, 0.0, 0.0
    

In [ ]:
import talib
dfPrices = get_pricing(symbols('AAPL'), start_date='2015-06-03', end_date='2016-02-03')
#dfPrices
plt.plot(dfPrices.index, dfPrices.price)
dfReturns = dfPrices.price - dfPrices.price.shift(1)
dfReturnsPrc = dfReturns / dfPrices.price.shift(1)*100
dfReturnsPrc.describe()

def createZigZagPoints(dfSeries, minSegSize=2, sizeInDevs=1):
    minRetrace = minSegSize
    curVal = dfSeries[0]
    curPos = dfSeries.index[0]
    curDir = 1
    #dfRes = pd.DataFrame(np.zeros((len(dfSeries.index), 2)), index=dfSeries.index, columns=["Dir","Value"])
    dfRes = pd.DataFrame(index=dfSeries.index, columns=["Dir", "Value"])
    #print(dfRes)
    #print(len(dfSeries.index))
    for ln in dfSeries.index:
        if((dfSeries[ln] - curVal)*curDir >= 0):
            curVal = dfSeries[ln]
            curPos = ln
    #print(str(ln) + ": moving curVal further, to " + str(curVal))
        else:
            retracePrc = abs((dfSeries[ln]-curVal)/curVal*100)
            #print(str(ln) + ": estimating retracePrc, it's " + str(retracePrc))
            if(retracePrc >= minRetrace):
            #print(str(ln) + ": registering key point, its pos is " + str(curPos) + ", value = " + str(curVal) + ",
                dir=" +str(curDir))
                dfRes.ix[curPos, 'Value'] = curVal
                dfRes.ix[curPos, 'Dir'] = curDir
                curVal = dfSeries[ln]
                curPos = ln
                curDir = -1*curDir
    #print(str(ln) + ": setting new cur vals, pos is " + str(curPos) + ", curVal = " + str(curVal) + ",dir=" +str(curDir))
    #print(ln, curVal, curDir)
    dfRes[['Value']] = dfRes[['Value']].astype(float)
    dfRes = dfRes.interpolate(method='linear')
    return(dfRes)

dfRes = createZigZagPoints(dfPrices.price)
#print(dfRes)
plt.plot(dfPrices['price'])
plt.plot(dfRes['Value'])